In [91]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

In [92]:
class Bess_Optimizer:
    def __init__(self, power_capacity = 100, energy_capacity = 200, efficiency = 0.9):
        self.power_capacity = power_capacity
        self.energy_capacity = energy_capacity
        self.energy_prices = None
        self.reg_up_prices = None
        self.reg_down_prices = None
        self.efficiency = efficiency

    def load_prices(self, energy_price_file):
        '''
        load energy prices from energy_price_file
        '''
        
        self.energy_prices = pd.read_csv(energy_price_file)
        self.energy_prices['Operating Day'] = pd.to_datetime(self.energy_prices['Operating Day'])
        self.energy_prices['Date'] = self.energy_prices['Operating Day'] + pd.to_timedelta(self.energy_prices['Operating Hour'] - 1, unit='h')
        self.energy_prices['Date'] = pd.to_datetime(self.energy_prices['Date'])
        self.energy_prices['Date'] = self.energy_prices['Date'].dt.strftime("%Y-%m-%d %H")
        
    def load_regulation(self):
        '''
        load regulation prices from regulation_price_file
        '''

    def optimize_period(self, start_day = None, end_day = None, initial_charge = 0):
        self.optimizer = LpProblem('Bess-Fluence', LpMaximize)                                              #set the model
        
        start_day = pd.to_datetime(start_day)
        end_day = pd.to_datetime(end_day)
        period = self.energy_prices[(self.energy_prices['Operating Day'] >= start_day) & (self.energy_prices['Operating Day'] <= end_day)]['Date'].tolist()
        

        #Decision Variables
        self.gen_hourly = LpVariable.dicts(                                                                 
                                    name = 'gen_hourly', 
                                    indices= period,
                                    lowBound = 0, 
                                    upBound = self.power_capacity
                                    )    

        self.charge_hourly = LpVariable.dicts(                                                                 
                                    name = 'charge_hourly', 
                                    indices= period,
                                    lowBound = 0, 
                                    upBound = self.power_capacity
                                    )     
        
        self.reg_up_power = LpVariable.dicts(
                                        name = 'reg_up_power',
                                        indices= period,
                                        lowBound = 0,
                                        upBound = self.power_capacity
                                        )
                                
        self.reg_down_power = LpVariable.dicts(
                                        name = 'reg_down_power',
                                        indices= period,
                                        lowBound = 0,
                                        upBound = self.power_capacity
                                        )

        self.total_charge = LpVariable.dicts(
                                        name = 'total_charge',
                                        indices= period,
                                        lowBound = 0,
                                        upBound = self.energy_capacity
                                        )
        

        # objective function
        self.optimizer += lpSum(
                                self.energy_prices[self.energy_prices['Date']==hour]['Price'].iloc[0] * self.gen_hourly[hour] -
                                self.energy_prices[self.energy_prices['Date']==hour]['Price'].iloc[0] * self.charge_hourly[hour] 
                                for hour in period
                                )
        
        #only n cycles of generation per day
        self.optimizer += lpSum(
                                self.gen_hourly[hour] 
                                for hour in period
                                ) <= self.energy_capacity
        
         #only n cycles of charege per day
        self.optimizer += lpSum(
                                self.charge_hourly[hour] 
                                for hour in period
                                ) <= self.energy_capacity
        
        self.optimizer += self.total_charge[period[0]] == initial_charge
        
        # temporary dependency on charginf for all hours
        for index in range(len(period)):
            if index > 0:
                past_hour = period[index-1]
                hour = period[index]

                self.optimizer += self.total_charge[hour] == self.total_charge[past_hour] + self.charge_hourly[past_hour] - self.gen_hourly[past_hour]
                self.optimizer += self.gen_hourly[hour] <= self.total_charge[hour] 
        
        self.optimizer.solve()


    def get_results(self):
        '''
        print the results of the optimization and the values of total profit
        '''
        print("Total Profit = ", self.optimizer.objective.value())
        for v in self.optimizer.variables():
            print(v.name, "=", v.varValue)






In [ ]:
optimizer = Bess_Optimizer()
optimizer.load_prices(energy_price_file ='data/energy_prices.csv')
optimizer.optimize_period(start_day = '1/1/2023', end_day = '1/1/2023', initial_charge = 50)
optimizer.get_results()
